In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+"haarcascade_frontalface_alt.xml")
cam = cv2.VideoCapture(0)

#keras_model_path = './callbacks_CK/emtion_model_CK.h5'
keras_model_path = '../code/callbacks/emtion_model.h5'
#emotion_labels = ['anger', 'disgust', 'Surprised',
#                'happy', 'sad','natural','afriad' ]
emotion_labels = ['anger', 'disgust', 'Surprised',
                'happy', 'sad','afriad' ,'natural']
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
restored_keras_model = tf.keras.models.load_model(keras_model_path)
print(restored_keras_model.metrics_names)

['loss', 'accuracy']


In [3]:
def EmtionDetect (img):
    face_area = img[y-h//4:y+h+h//4, x-w//4:x+w+w//4]
    face_area_gray = cv2.cvtColor(face_area, cv2.COLOR_BGR2GRAY)
    face_area_gray = cv2.cvtColor(face_area_gray, cv2.COLOR_GRAY2BGR)
    face_area_gray = cv2.resize(face_area_gray,(48, 48))
    face_area_gray = face_area_gray/255
    array_img = image.img_to_array(face_area_gray)
    array_img = np.expand_dims(array_img, axis=0)
    emotion_prediction = restored_keras_model(array_img)
    #print(emotion_prediction)
    emotionprobability = np.max(emotion_prediction)
    emotion_label_arg = np.argmax(emotion_prediction)
    emotion_text = emotion_labels[emotion_label_arg]  
    return emotion_text

In [4]:
while(True):
    ret, frame = cam.read()
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    img = frame
    for (x,y,w,h) in faces:
        img = cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        if (w>0) and (h>0):
            predict_result = EmtionDetect(img)
        cv2.putText(img=frame, text=predict_result, org=(x, y), fontFace=font, color=(255,255,255), fontScale = 2)
    cv2.imshow('frame',img)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()